In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cheese=pd.read_csv("/content/drive/My Drive/Data/cheese.csv")

In [ ]:
cheese

,Unnamed: 0,RETAILER,VOLUME,DISP,PRICE
0,1,LOS ANGELES - LUCKY,21374,0.162000,2.578460
1,2,LOS ANGELES - RALPHS,6427,0.124113,3.727867
2,3,LOS ANGELES - VONS,17302,0.102000,2.711421
3,4,CHICAGO - DOMINICK,13561,0.027591,2.651206
4,5,CHICAGO - JEWEL,42774,0.090613,1.986674
...,...,...,...,...,...
5550,5551,SYRACUSE - WEGMANS,751,0.280321,3.338216
5551,5552,NEW ENGLAND (NORTH) - SHOP N SAVE,5031,0.094000,2.379845
5552,5553,BIRMINGHAM/MONTGOM - KROGER,1400,0.000000,2.654286
5553,5554,NEW YORK (NEW) - A & P,3448,0.043846,3.524942


### Correlation Matrix



In [ ]:
cheese[['VOLUME','DISP','PRICE']].corr(method ='pearson') 

,VOLUME,DISP,PRICE
VOLUME,1.000000,0.172569,-0.226818
DISP,0.172569,1.000000,-0.148625
PRICE,-0.226818,-0.148625,1.000000


In [ ]:
cheese

,Unnamed: 0,RETAILER,VOLUME,DISP,PRICE
0,1,LOS ANGELES - LUCKY,21374,0.162000,2.578460
1,2,LOS ANGELES - RALPHS,6427,0.124113,3.727867
2,3,LOS ANGELES - VONS,17302,0.102000,2.711421
3,4,CHICAGO - DOMINICK,13561,0.027591,2.651206
4,5,CHICAGO - JEWEL,42774,0.090613,1.986674
...,...,...,...,...,...
5550,5551,SYRACUSE - WEGMANS,751,0.280321,3.338216
5551,5552,NEW ENGLAND (NORTH) - SHOP N SAVE,5031,0.094000,2.379845
5552,5553,BIRMINGHAM/MONTGOM - KROGER,1400,0.000000,2.654286
5553,5554,NEW YORK (NEW) - A & P,3448,0.043846,3.524942


### Seperating the City and Company from Retailer.

In [ ]:
Company=[]
city=[]
for retailer in cheese["RETAILER"].values:
  city.append(retailer.split("-")[0])
  Company.append(retailer.split("-")[1])

### Tabulating the cities and the companies in the cheese dataset

In [ ]:
cheese["CITY"]=city
cheese["Company"]=Company

In [ ]:
cheese

,Unnamed: 0,RETAILER,VOLUME,DISP,PRICE,CITY,Company
0,1,LOS ANGELES - LUCKY,21374,0.162000,2.578460,LOS ANGELES,LUCKY
1,2,LOS ANGELES - RALPHS,6427,0.124113,3.727867,LOS ANGELES,RALPHS
2,3,LOS ANGELES - VONS,17302,0.102000,2.711421,LOS ANGELES,VONS
3,4,CHICAGO - DOMINICK,13561,0.027591,2.651206,CHICAGO,DOMINICK
4,5,CHICAGO - JEWEL,42774,0.090613,1.986674,CHICAGO,JEWEL
...,...,...,...,...,...,...,...
5550,5551,SYRACUSE - WEGMANS,751,0.280321,3.338216,SYRACUSE,WEGMANS
5551,5552,NEW ENGLAND (NORTH) - SHOP N SAVE,5031,0.094000,2.379845,NEW ENGLAND (NORTH),SHOP N SAVE
5552,5553,BIRMINGHAM/MONTGOM - KROGER,1400,0.000000,2.654286,BIRMINGHAM/MONTGOM,KROGER
5553,5554,NEW YORK (NEW) - A & P,3448,0.043846,3.524942,NEW YORK (NEW),A & P


### Listing out the distinct Cities and Companies in the data.

In [ ]:
cities=list(set(cheese["CITY"].values))

In [ ]:
len(cities)

46

In [ ]:
companies=list(set(cheese["Company"].values))

In [ ]:
len(companies)

50

### Assuming 1.8 USD is the per unit cost of cheese production

In [ ]:
from sklearn.linear_model import LinearRegression 

# City level analysis

### The function following outputs the city , their profitability (increase / decrease) and their revenue change with respect to the alternating price.




In [ ]:
def discount_check_city(data,city,discount):
    model=LinearRegression()
    train=data.loc[data['CITY'] == city]
    model.fit(np.array(train['PRICE']).reshape(-1,1),np.array(train['VOLUME']).reshape(-1,1))# Regression model Price vs Volume
    observed_revenue=sum([row[0]*row[1] for index,row in train[["VOLUME","PRICE"]].iterrows()])# Total observed revenue from the city
    observed_profit=observed_revenue-1.8*sum(train["VOLUME"])# Total observed profit from the city
    alternate_price=[i-i*discount for i in train["PRICE"]]#Listing out  alternate prices after discounting
    test=pd.DataFrame()
    test["alternate_price"]=alternate_price
    predicted_volume=model.predict(test)# Predicted volumes corresponding to the alternate prices
    alternate_revenue=sum([i[0]*i[1] for i in zip(predicted_volume,alternate_price)])# Total predicted alternate revenue from the city
    alternate_profit=alternate_revenue-1.8*sum(predicted_volume)# Total predicted profit from alternate pricing from the city
    # Profitability comaparison
    if alternate_profit-observed_profit>0:
      return [city,"profit_increase",alternate_revenue-observed_revenue,alternate_profit-observed_profit]
    else:
      return [city,"profit_decrease",alternate_revenue-observed_revenue,alternate_profit-observed_profit]         

### Performing the experiment with 5% discount for each of the listed cities.

In [ ]:
profit_increase_city=[]
profit_decrease_city=[]
for city in cities:
  output=discount_check_city(cheese,city,discount=0.05)
  if output[1]=="profit_increase":
    profit_increase_city.append(output)
  else:
    profit_decrease_city.append(output)  

In [ ]:
positive_profitability_city=pd.DataFrame()
positive_profitability_city["CITY"]=[i[0] for i in profit_increase_city]
positive_profitability_city["Profitability"]=[round(i[3][0],4) for i in profit_increase_city]
positive_profitability_city["Revenue change"]=[round(i[2][0],4) for i in profit_increase_city]

### DataFrame tabulating the Cities showing increase in profitability after 5% discounting.

In [ ]:
positive_profitability_city

,CITY,Profitability,Revenue change
0,LOS ANGELES,36756.1995,5.139325e+05
1,BALTI/WASH,52514.6204,2.351523e+05
2,SYRACUSE,37559.7042,1.693369e+05
3,DENVER,11063.8487,1.126076e+05
4,PHOENIX,10967.7746,1.064757e+05
5,NEW YORK (NEW),413262.6861,9.120110e+05
6,CHICAGO,300378.9614,1.520790e+06
7,"ALBANY,NY",16235.6532,7.237576e+04
8,MILWAUKEE,679.4523,3.068904e+04
9,ST. LOUIS,131055.5894,5.123098e+05


### Sorted table(in decreasing order of profitability).

In [ ]:
positive_profitability_city.sort_values("Profitability", axis = 0, ascending = False) 

,CITY,Profitability,Revenue change
5,NEW YORK (NEW),413262.6861,9.120110e+05
12,BUFFALO/ROCHESTER,386188.8888,1.145224e+06
6,CHICAGO,300378.9614,1.520790e+06
9,ST. LOUIS,131055.5894,5.123098e+05
15,PHILADELPHIA,123250.6130,3.406509e+05
1,BALTI/WASH,52514.6204,2.351523e+05
14,SAN FRANCISCO,46093.5697,2.025989e+05
13,HARTFORD,44715.3028,1.442358e+05
2,SYRACUSE,37559.7042,1.693369e+05
0,LOS ANGELES,36756.1995,5.139325e+05


In [ ]:
negative_profitability_city=pd.DataFrame()
negative_profitability_city["CITY"]=[i[0] for i in profit_decrease_city]
negative_profitability_city["Profitability"]=[round(i[3][0],4) for i in profit_decrease_city]
negative_profitability_city["Revenue change"]=[round(i[2][0],4) for i in profit_decrease_city]

### DataFrame tabulating the Cities showing decrease in profitability after 5% discounting.

In [ ]:
negative_profitability_city

,CITY,Profitability,Revenue change
0,ATLANTA,-37892.5910,23067.5704
1,NASHVILLE,-22020.7760,11027.2917
2,ROANOKE (NEW),-26940.4892,80693.5874
3,SAN ANT/CORPUS CHR,-35843.6700,136331.4942
4,CHARLOTTE,-82883.1122,-93689.9409
5,TAMPA/ST. PETE,-45026.3437,111427.5788
6,NEW ENGLAND (NORTH),-33847.3307,59887.1097
7,DALLAS/FT. WORTH,-26837.0884,141778.6953
8,"JACKSONVILLE,FL",-29248.3562,23913.3354
9,HOUSTON,-22239.1263,158988.3158


### Sorted table(in decreasing order of profitability).

In [ ]:
negative_profitability_city.sort_values("Profitability", axis = 0, ascending = False) 

,CITY,Profitability,Revenue change
17,"COLUMBUS,OH",-949.8203,112065.3526
15,OKLAHOMA CITY,-2287.7427,42428.0513
24,WICHITA,-3939.4150,9368.9065
18,HARRISBURG/SCRANTN,-7184.0004,6537.3530
10,LITTLE ROCK,-7307.0615,20033.6957
26,CLEVELAND,-9404.9648,37362.3857
27,LOUISVILLE,-11828.1941,71788.1460
22,NEW ORLEANS,-12344.5229,46332.1063
14,CINCINNATI,-14198.4295,66573.0211
20,PITTSBURGH,-14407.1898,18421.0389


### Overall profitability from city level analysis (country wide).

In [ ]:
sum(positive_profitability_city["Profitability"])+sum(negative_profitability_city["Profitability"])

680750.5824000001

### Overall revenue change city level analysis (country wide).

In [ ]:
sum(positive_profitability_city["Revenue change"])+sum(negative_profitability_city["Revenue change"])

7591345.6294

# Company level analysis

### The function following outputs the company , their profitability (increase / decrease) and their revenue change with respect to the alternating price.

In [ ]:
def discount_check_company(data,company,discount):
    model=LinearRegression()
    train=data.loc[data['Company'] ==company]
    model.fit(np.array(train['PRICE']).reshape(-1,1),np.array(train['VOLUME']).reshape(-1,1))# Regression model Price vs Volume
    observed_revenue=sum([row[0]*row[1] for index,row in train[["VOLUME","PRICE"]].iterrows()])# Total observed revenue from the company
    observed_profit=observed_revenue-1.8*sum(train["VOLUME"])# Total observed profit from the company
    alternate_price=[i-i*discount for i in train["PRICE"]]# Listing out alternate pricing after discounting
    test=pd.DataFrame()
    test["alternate_price"]=alternate_price
    predicted_volume=model.predict(test)# Predicted volumes corresponding to the alternate prices
    alternate_revenue=sum([i[0]*i[1] for i in zip(predicted_volume,alternate_price)])# Total predicted alternate revenue from the company
    alternate_profit=alternate_revenue-1.8*sum(predicted_volume)# Total predicted profit from alternate pricing from the company
    # Profitability comparison
    if alternate_profit-observed_profit>0:
      return [company,"profit_increase",alternate_revenue-observed_revenue,alternate_profit-observed_profit]
    else:
      return [company,"profit_decrease",alternate_revenue-observed_revenue,alternate_profit-observed_profit]                   

### Performing the experiment with 5% discount for each of the listed companies.

In [ ]:
profit_increase_company=[]
profit_decrease_company=[]
for company in companies:
  output=discount_check_company(cheese,company,discount=0.05)
  if output[1]=="profit_increase":
    profit_increase_company.append(output)
  else:
    profit_decrease_company.append(output) 

In [ ]:
positive_profitability_company=pd.DataFrame()
positive_profitability_company["Company"]=[i[0] for i in profit_increase_company]
positive_profitability_company["Profitability"]=[round(i[3][0],4) for i in profit_increase_company]
positive_profitability_company["Revenue change"]=[round(i[2][0],4) for i in profit_increase_company]

### DataFrame tabulating the Companies showing increase in profitability after 5% discounting.

In [ ]:
positive_profitability_company

,Company,Profitability,Revenue change
0,FARMER JACKS,25105.2797,164807.1016
1,PATHMARK,303293.8737,625738.2659
2,RANDALLS,5211.4815,79494.6298
3,BIG BEAR,222.1421,38865.5602
4,STAR MARKET,9714.8275,45513.0282
5,PRICE CHOPPER,32189.3656,140074.7105
6,LUCKY,15493.4850,431871.7031
7,SUPER FRESH,20684.1625,60848.7870
8,DOMINICK,250237.3960,822009.4892
9,TOPS MARKETS,212534.7510,638548.2151


### Sorted table(in decreasing order of profitability).

In [ ]:
positive_profitability_company.sort_values("Profitability", axis = 0, ascending = False) 

,Company,Profitability,Revenue change
1,PATHMARK,303293.8737,625738.2659
8,DOMINICK,250237.3960,822009.4892
9,TOPS MARKETS,212534.7510,638548.2151
15,WEGMANS,192489.2617,571577.0358
20,JEWEL,190991.0728,849490.0623
22,ACME MARKET,123250.6130,340650.8959
24,GIANT FOOD INC,85476.5592,232351.6087
23,VONS,81990.6394,287006.1118
18,OMNI,66941.0815,364538.8831
14,NATIONAL SUPER,66365.1920,203814.8950


In [ ]:
negative_profitability_company=pd.DataFrame()
negative_profitability_company["Company"]=[i[0] for i in profit_decrease_company]
negative_profitability_company["Profitability"]=[round(i[3][0],4) for i in profit_decrease_company]
negative_profitability_company["Revenue change"]=[round(i[2][0],4) for i in profit_decrease_company]

### DataFrame tabulating the Companies showing decrease in profitability after 5% discounting.

In [ ]:
negative_profitability_company

,Company,Profitability,Revenue change
0,SMITHS FOOD,-22723.2378,-4287.5104
1,KROGER,-12232.8569,-486.5553
2,HOMELAND,-2287.7427,42428.0513
3,RALPHS,-10566.0582,127442.9040
4,HARVEST FOODS,-7307.0615,20033.6957
5,BI LO,-194589.8216,-673330.8739
6,FOOD LION,-155629.6512,-46742.9234
7,NEW FARM FRESH,-15848.5552,49.9585
8,KASH N KARRY,-20514.4755,3700.3589
9,GIANT FOOD STO,-7184.0004,6537.3530


### Sorted table(in decreasing order of profitability).

In [ ]:
negative_profitability_company.sort_values("Profitability", axis = 0, ascending = False) 

,Company,Profitability,Revenue change
21,STOP & SHOP,-1787.9097,82289.6615
2,HOMELAND,-2287.7427,42428.0513
22,PUBLIX,-3921.3626,389615.2217
12,DILLON COMPANIES,-3939.4150,9368.9065
9,GIANT FOOD STO,-7184.0004,6537.3530
4,HARVEST FOODS,-7307.0615,20033.6957
19,HARRIS TEETER,-8348.1985,6661.1798
11,STOP N SHOP,-9404.9648,37362.3857
3,RALPHS,-10566.0582,127442.9040
1,KROGER,-12232.8569,-486.5553


### Overall profitability from company level analysis (country wide).


In [ ]:
sum(positive_profitability_company["Profitability"])+sum(negative_profitability_company["Profitability"])

646486.3475999997

### Overall Revenue change from company level analysis (country wide).





In [ ]:
sum(positive_profitability_company["Revenue change"])+sum(negative_profitability_company["Revenue change"])

7282689.055900002

# Retailer level analysis

### Listing out the retailers.

In [ ]:
Retailers=list(set(cheese["RETAILER"]))

In [ ]:
len(Retailers)

88

### The function following outputs the retailer , their profitability (increase / decrease) and their revenue change with respect to the alternating price.

In [ ]:
def discount_check_retailer(data,retailer,discount):
    model=LinearRegression()
    train=data.loc[data['RETAILER'] == retailer]
    model.fit(np.array(train['PRICE']).reshape(-1,1),np.array(train['VOLUME']).reshape(-1,1))# Regression model Price vs Volume
    observed_revenue=sum([row[0]*row[1] for index,row in train[["VOLUME","PRICE"]].iterrows()])# Total observed revenue from the retailer
    observed_profit=observed_revenue-1.8*sum(train["VOLUME"])# Total observed profit from the retailer
    alternate_price=[i-i*discount for i in train["PRICE"]]# Listing out  alternate prices after discounting
    test=pd.DataFrame()
    test["alternate_price"]=alternate_price
    predicted_volume=model.predict(test)# Predicted volumes corresponding to the alternate prices
    alternate_revenue=sum([i[0]*i[1] for i in zip(predicted_volume,alternate_price)])# Total predicted alternate revenue from the retailer
    alternate_profit=alternate_revenue-1.8*sum(predicted_volume)# Total predicted profit from alternate pricing from the retailer
    # Profitability comaparison
    if alternate_profit-observed_profit>0:
      return [retailer,"profit_increase",alternate_revenue-observed_revenue,alternate_profit-observed_profit]
    else:
      return [retailer,"profit_decrease",alternate_revenue-observed_revenue,alternate_profit-observed_profit]         

### Performing the experiment with 5% discount for each of the listed retailers.

In [ ]:
profit_increase_retailer=[]
profit_decrease_retailer=[]
for retailer in Retailers:
  output=discount_check_retailer(cheese,retailer,discount=0.05)
  if output[1]=="profit_increase":
    profit_increase_retailer.append(output)
  else:
    profit_decrease_retailer.append(output) 

In [ ]:
positive_profitability_retailer=pd.DataFrame()
positive_profitability_retailer["RETAILER"]=[i[0] for i in profit_increase_retailer]
positive_profitability_retailer["Profitability"]=[round(i[3][0],4) for i in profit_increase_retailer]
positive_profitability_retailer["Revenue change"]=[round(i[2][0],4) for i in profit_increase_retailer]

### DataFrame tabulating the retailer showing increase in profitability after 5% discounting.




In [ ]:
positive_profitability_retailer

,RETAILER,Profitability,Revenue change
0,BOSTON - STAR MARKET,9714.8275,45513.0282
1,SAN FRANCISCO - LUCKY,46093.5697,202598.9311
2,SACRAMENTO - RALEYS,11296.5235,50518.1178
3,MILWAUKEE - KOHLS FOOD STORE,679.4523,30689.0371
4,DENVER - KING SOOPERS INC,11063.8487,112607.6270
5,BALTI/WASH - GIANT FOOD INC,85476.5592,232351.6087
6,ST. LOUIS - NATIONAL SUPER,66365.1920,203814.8950
7,PHILADELPHIA - ACME MARKET,123250.6130,340650.8959
8,SYRACUSE - PRICE CHOPPER,15468.1888,66108.5778
9,BALTI/WASH - SAFEWAY,28066.2119,107967.4289


### Sorted table(in decreasing order of profitability).

In [ ]:
positive_profitability_retailer.sort_values("Profitability", axis = 0, ascending = False) 

,RETAILER,Profitability,Revenue change
15,NEW YORK (NEW) - PATHMARK,303293.8737,625738.2659
23,CHICAGO - DOMINICK,250237.3960,822009.4892
19,BUFFALO/ROCHESTER - TOPS MARKETS,212534.7510,638548.2151
33,CHICAGO - JEWEL,190991.0728,849490.0623
20,BUFFALO/ROCHESTER - WEGMANS,161327.1637,479592.2345
7,PHILADELPHIA - ACME MARKET,123250.6130,340650.8959
26,SOUTH CAROLINA - WINN DIXIE,108060.2431,633654.5367
5,BALTI/WASH - GIANT FOOD INC,85476.5592,232351.6087
10,LOS ANGELES - VONS,81990.6394,287006.1118
18,CHICAGO - OMNI,66941.0815,364538.8831


In [ ]:
negative_profitability_retailer=pd.DataFrame()
negative_profitability_retailer["Company"]=[i[0] for i in profit_decrease_retailer]
negative_profitability_retailer["Profitability"]=[round(i[3][0],4) for i in profit_decrease_retailer]
negative_profitability_retailer["Revenue change"]=[round(i[2][0],4) for i in profit_decrease_retailer]

### DataFrame tabulating the retailer showing decrease in profitability after 5% discounting.


In [ ]:
negative_profitability_retailer

,Company,Profitability,Revenue change
0,CINCINNATI - KROGER CO,-14198.4295,66573.0211
1,DETROIT - KROGER CO,-13262.5653,45789.4288
2,BIRMINGHAM/MONTGOM - WINN DIXIE,-23508.2508,38433.5886
3,PITTSBURGH - GIANT EAGLE,-14407.1898,18421.0389
4,ROANOKE (NEW) - KROGER CO,-18738.5425,52895.0510
5,DALLAS/FT. WORTH - ALBERTSONS,-12431.7880,25946.6795
6,RALEIGH/GREENSBORO - WINN DIXIE,-6810.2752,30766.6681
7,LITTLE ROCK - HARVEST FOODS,-7307.0615,20033.6957
8,"ORLANDO,FL - WINN DIXIE",-25504.3107,3965.7375
9,CLEVELAND - STOP N SHOP,-9404.9648,37362.3857


### Sorted table(in decreasing order of profitability).

In [ ]:
negative_profitability_retailer.sort_values("Profitability", axis = 0, ascending = False) 

,Company,Profitability,Revenue change
19,DALLAS/FT. WORTH - WINN DIXIE,-1990.7162,37386.1242
50,SAN DIEGO - RALPHS,-2018.0414,30398.2758
22,OKLAHOMA CITY - HOMELAND,-2287.7427,42428.0513
51,DALLAS/FT. WORTH - KROGER CO,-2875.1960,51600.6488
35,WICHITA - DILLON COMPANIES,-3939.4150,9368.9065
37,"JACKSONVILLE,FL - PUBLIX",-4906.3681,14693.6253
45,"ORLANDO,FL - FOOD LION",-5745.6465,-1261.0024
39,LOUISVILLE - WINN DIXIE,-6495.5157,20073.2442
34,PHOENIX - SMITHS FOOD,-6808.1862,6306.5050
6,RALEIGH/GREENSBORO - WINN DIXIE,-6810.2752,30766.6681


### Overall profitability from retailer level analysis (country wide).


In [ ]:
sum(positive_profitability_retailer["Profitability"])+sum(negative_profitability_retailer["Profitability"])

1086439.7551000002

### Overall Revenue change from retailer level analysis (country wide).


In [ ]:
sum(positive_profitability_retailer["Revenue change"])+sum(negative_profitability_retailer["Revenue change"])

8828100.2698